In [17]:
import requests

def check_match(name):
    base_url = "http://rest.genenames.org/search/symbol:"
    r = requests.get(base_url+name, headers={'Accept': 'application/json'})
    return r.json()['response']['numFound']

In [20]:
!ls

3_TR_Molecule_Name_Disambigation.ipynb
4_TR_Graph_Viz_Molecule_Maps.ipynb
DO_NOT_GIT_secret
README.md
TR_1 metapub module for downloading abstracts from pubmed.ipynb
TR_2_Clustered_Heatmap.ipynb
TR_RNAi_test.png
Untitled.ipynb
images
letter_graph.png
letter_graph2.png
pickles
simple.dot
simple.png
star.dot
star.png


In [92]:
with open('pickles/best_candidates_sorted.txt', 'r') as f:
    read_data = f.read()
genes = read_data.split('\n')

In [93]:
for g in genes[0:20]:
    print(g)

A-GAMMA3'E
A1BG
A1BG-AS1
A1CF
A2M
A2M-AS1
A2ML1
A2ML1-AS1
A2ML1-AS2
A2MP1
A3GALT2
A4GALT
A4GNT
AA06
AA1
AA2
AAA1
AAA2
AAA3
AAA4


In [136]:
def urls(genes):
    collection =[]
    for a_gene in genes:
        base_url = "http://rest.genenames.org/search/symbol:"
        url = base_url+a_gene
        collection.append(url)
    return collection

In [144]:
urls_ = urls(genes)

In [145]:
len(urls_)

61128

In [146]:
# Setting up the Queue

from threading import Thread
import queue
import logging
import urllib.request as urlreq
import time
#import urllib.errors

#urls_ = urls_[0:100]

logging.basicConfig(format='%(message)s')

logging.warning('I print to stderr by default')
#logging.info('For this you must change the level and add a handler.')


#set up the queue to hold all the urls
q = queue.Queue(maxsize=0)
# Use many threads (50 max, or one for each url)
num_theads = min(10, len(urls_))

##Populating Queue with tasks
results = [{} for x in urls_];
##load up the queue with the urls to fetch and the index for each job (as a tuple):
for i in range(len(urls_)):
    #need the index and the url in each queue item.
    q.put((i,urls_[i]))
 
    
# Threaded function for queue processing.
def crawl(q, result):
    while not q.empty():
        time.sleep(1)
        #base_url = "http://rest.genenames.org/search/symbol:"
        #url = base_url+a_gene 
        
        work = q.get()                      #fetch new work from the Queue
        try:
            data = requests.get(work[1], headers={'Accept': 'application/json'})
            logging.info("Requested..." + work[1])
            result[work[0]]= (work[1].replace('http://rest.genenames.org/search/symbol:',''), data.json()['response']['numFound'])         #Store data back at correct index
            if work[0]%100==0:
                logging.warning(work[0])
        except:
            logging.error('Error with URL check!')
            result[work[0]] = {}
        #signal to the queue that task has been processed
        q.task_done()
    return True    

##Starting worker threads on queue processing
for i in range(num_theads):
    logging.debug('Starting thread ', i)
    worker = Thread(target=crawl, args=(q,results))
    worker.setDaemon(True)    #setting threads as "daemon" allows main program to 
                              #exit eventually even if these dont finish 
                              #correctly.
    worker.start()
#now we wait until the queue has been processed
q.join()
logging.warning('all tasks completed')

ERROR:root:Error with URL check!


In [157]:
results[60000]


('YAM1', 1)

In [148]:
#be sure to pickle the resutls variable 
approved=0
total=0
for i in range(10):
    total+=1
    if results[i][1] >=1:
        approved+=1 
        #print(results[i])
print(approved/total)

0.9


In [151]:
import pickle

pickle.dump( results, open( "nomenclature_approved.pkl", "wb" ) )

In [153]:
win = pickle.load( open( "nomenclature_approved.pkl", "rb" ) )

In [156]:
!open .